In [1]:
!pip -q install torch_geometric rectools
!pip -q install comet_ml
!pip -q install python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 86.8 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.1/727.1 kB 12.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 55.0 MB/s eta 0:00:00a 0:00:01


In [2]:
import comet_ml
from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model

from dotenv import load_dotenv
import os

In [ ]:
load_dotenv(".env")

True

In [4]:
experiment = Experiment(
  api_key=os.getenv('API_KEY'),
  project_name="gnn-recommender",
  workspace="annanet",
  log_code=True
)

experiment.set_name('diffGATGraphModel-books')
experiment.add_tags(['books', 'leave-n-out'])

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/annanet/gnn-recommender/cf72e82fe89b46d282db2a35def476b3

COMET INFO: Couldn't find a Git repository in '/kaggle/working' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.


https://www.comet.com/annanet/gnn-recommender/cf72e82fe89b46d282db2a35def476b3

In [5]:
hyperparameters = {
    'seed': 42,
    'types_of_feedback': ["explicit_positive", "expliсit_negative",
                          "implicit_positive", "implicit_negative"],
    'train_edge_type': ('item','to_feedback_explicit_positive','explicit_positive'),
    'train_num_epochs': 100,
    'train_lr': 8e-5,
    'train_batch_size': 32768,
    'train_print_every': 10,  
    'train_test_every': 10,
    'test_topk': 10,
    'test_batch_size': 32768
}

In [6]:
import os
os.listdir('/kaggle/input/data/leave-n-out/books')

['train.csv', 'test.csv']

In [7]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import HeteroData
from torch_geometric.nn import HeteroConv, SAGEConv, GATConv

from sklearn.preprocessing import LabelEncoder

from rectools import Columns
from rectools.metrics import MAP, Precision, Recall, NDCG, calc_metrics

import gc
import random

In [8]:
SEED = hyperparameters['seed']
torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)

In [9]:
rootpath = '/kaggle/input/data/leave-n-out/books/'
train = pd.read_csv(
    rootpath+'train.csv'
)
train['date'] = pd.to_datetime(train['date_added'])
print(train.head())

                            user_id  book_id  rating  \
0  000883382802f2d95a3dd545bb953882  8525590       3   
1  000883382802f2d95a3dd545bb953882  2767052       5   
2  000883382802f2d95a3dd545bb953882  3236307       5   
3  000883382802f2d95a3dd545bb953882   256683       4   
4  000883382802f2d95a3dd545bb953882  6001758       5   

                  date_added                       date  
0  2011-10-31 18:37:56-07:00  2011-10-31 18:37:56-07:00  
1  2011-10-31 18:40:33-07:00  2011-10-31 18:40:33-07:00  
2  2011-11-02 08:30:30-07:00  2011-11-02 08:30:30-07:00  
3  2011-11-02 08:55:16-07:00  2011-11-02 08:55:16-07:00  
4  2011-11-02 08:57:21-07:00  2011-11-02 08:57:21-07:00  


/tmp/ipykernel_35/4238775169.py:5: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  train['date'] = pd.to_datetime(train['date_added'])


In [10]:
explicit_positive = train[(train["rating"] == 5)].index
explisit_negative = train[(train["rating"] <= 2)].index

explicit_combined_feedback = explicit_positive.union(explisit_negative)
print('Количество explicit позитивного фидбека', explicit_positive.shape[0])
print('Количество explicit негативного фидбека', explisit_negative.shape[0])

Количество explicit позитивного фидбека 359463
Количество explicit негативного фидбека 178217


In [11]:
implicit_positive = train[(train["rating"] == 4)].index
implicit_negative = train[(train["rating"] == 3)].index

implicit_combined_feedback = implicit_positive.union(implicit_negative)
print('Количество implicit позитивного фидбека', implicit_positive.shape[0])
print('Количество implicit негативного фидбека', implicit_negative.shape[0])

Количество implicit позитивного фидбека 430342
Количество implicit негативного фидбека 258651


In [12]:
train.loc[:, "target"] = ""
train.loc[explicit_positive, "target"] = "explicit_positive"
train.loc[explisit_negative, "target"] = "expliсit_negative"
train.loc[implicit_positive, "target"] = "implicit_positive"
train.loc[implicit_negative, "target"] = "implicit_negative"

train = train[['user_id','book_id','target','date']]
train.head()

,user_id,book_id,target,date
0,000883382802f2d95a3dd545bb953882,8525590,implicit_negative,2011-10-31 18:37:56-07:00
1,000883382802f2d95a3dd545bb953882,2767052,explicit_positive,2011-10-31 18:40:33-07:00
2,000883382802f2d95a3dd545bb953882,3236307,explicit_positive,2011-11-02 08:30:30-07:00
3,000883382802f2d95a3dd545bb953882,256683,implicit_positive,2011-11-02 08:55:16-07:00
4,000883382802f2d95a3dd545bb953882,6001758,explicit_positive,2011-11-02 08:57:21-07:00


In [13]:
train = train.sort_values(by=["user_id", "date"]).reset_index(drop=True)
train.columns = ['user_id', 'item_id', 'target', 'date']

In [14]:
test = pd.read_csv(
    rootpath+'test.csv'
)
test['date'] = pd.to_datetime(test['date_added'])
print(test.head())

                            user_id   book_id  rating  \
0  000883382802f2d95a3dd545bb953882   8135807       4   
1  000883382802f2d95a3dd545bb953882  18301124       5   
2  000883382802f2d95a3dd545bb953882  18220354       4   
3  000883382802f2d95a3dd545bb953882  17383918       5   
4  000883382802f2d95a3dd545bb953882  13188676       5   

                  date_added                       date  
0  2013-08-13 09:37:39-07:00  2013-08-13 09:37:39-07:00  
1  2013-10-27 22:18:01-07:00  2013-10-27 22:18:01-07:00  
2  2013-12-09 22:20:59-08:00  2013-12-09 22:20:59-08:00  
3  2013-12-22 20:57:14-08:00  2013-12-22 20:57:14-08:00  
4  2013-12-22 20:58:15-08:00  2013-12-22 20:58:15-08:00  


/tmp/ipykernel_35/2717431719.py:4: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  test['date'] = pd.to_datetime(test['date_added'])


In [15]:
test = test[['user_id','book_id', 'date']]
test.columns = ['user_id', 'item_id', 'date']
test.head()

,user_id,item_id,date
0,000883382802f2d95a3dd545bb953882,8135807,2013-08-13 09:37:39-07:00
1,000883382802f2d95a3dd545bb953882,18301124,2013-10-27 22:18:01-07:00
2,000883382802f2d95a3dd545bb953882,18220354,2013-12-09 22:20:59-08:00
3,000883382802f2d95a3dd545bb953882,17383918,2013-12-22 20:57:14-08:00
4,000883382802f2d95a3dd545bb953882,13188676,2013-12-22 20:58:15-08:00


# MVP model v2

In [16]:
test = test[(test.user_id.isin(train.user_id)) & (test.item_id.isin(train.item_id))].copy()
test.shape

(151126, 3)

In [17]:
# 2. Преобразование данных - для куарека не особо нужно, но для других - напоминалка
# делаем всегда! чтобы не сломать ничего дальше и чтобы все индексы были от 0 до N без пропусков
user_encoder = LabelEncoder()
video_encoder = LabelEncoder()

train.loc[:, 'user_id'] = user_encoder.fit_transform(train['user_id'])
train.loc[:, 'item_id'] = video_encoder.fit_transform(train['item_id'])

test.loc[:, 'user_id'] = user_encoder.transform(test['user_id'])
test.loc[:, 'item_id'] = video_encoder.transform(test['item_id'])

train['user_id'] = train['user_id'].astype(int)
train['item_id'] = train['item_id'].astype(int)
test['user_id'] = test['user_id'].astype(int)
test['item_id'] = test['item_id'].astype(int)

In [18]:
# т.е. сразу знаем количество и в каких пределах изменяется user_id и video_id
num_videos = train['item_id'].nunique()
num_users = train['user_id'].nunique()

print('Количество уникальных item_id', num_videos)
print('Количество уникальных user_id', num_users)

Количество уникальных item_id 25456
Количество уникальных user_id 18892


In [19]:
def prepare_hetero_data(df) -> HeteroData:
    data = HeteroData()
    users = df['user_id'].unique()
    num_users = len(users)
    num_items = df['item_id'].max() + 1
    feedback_types = df['target'].unique().tolist()

    # узлы
    data['user'].node_id = torch.arange(num_users)
    data['item'].node_id = torch.arange(num_items)
    for ft in feedback_types:
        data[ft].node_id = torch.arange(num_users)

    # ребра item<->feedback->user
    for ft in feedback_types:
        mask = df['target'] == ft
        items = torch.LongTensor(df.loc[mask, 'item_id'].values)
        users_idx = torch.LongTensor(df.loc[mask, 'user_id'].values)
        # item -> fb
        data['item', f'to_feedback_{ft}', ft].edge_index = torch.stack([items, users_idx], dim=0)
        # fb -> item
        data[ft, f'feedback_to_item_{ft}', 'item'].edge_index = torch.stack([users_idx, items], dim=0)
        # fb -> user (1:1)
        idx = torch.arange(num_users)
        data[ft, f'to_user_{ft}', 'user'].edge_index = torch.stack([idx, idx], dim=0)

    return data

In [20]:
class SignedHeteroGNN(nn.Module):
    def __init__(self,
                 num_users: int,
                 num_items: int,
                 feedback_types: list,
                 emb_dim: int = 32,
                 hidden_dim: int = 16,
                 heads: int = 2,
                 dropout: float = 0.2):
        super().__init__()
        self.feedback_types = feedback_types
        self.pos_types = ['implicit_positive', 'explicit_positive']
        self.neg_types = ['implicit_negative', 'expliсit_negative']

        self.user_emb = nn.Embedding(num_users, emb_dim)
        self.item_emb = nn.Embedding(num_items, emb_dim)
        self.fb_emb   = nn.ModuleDict({
            ft: nn.Embedding(num_users, emb_dim) for ft in feedback_types
        })

        conv1, conv2 = {}, {}
        for ft in feedback_types:
            # ft -> user
            conv1[(ft, f'to_user_{ft}', 'user')] = GATConv(emb_dim, hidden_dim,
                                                      heads=heads,
                                                      add_self_loops=False)
            conv2[(ft, f'to_user_{ft}', 'user')] = GATConv(hidden_dim*heads, emb_dim,
                                                      heads=1,
                                                      add_self_loops=False)
            # ft -> item
            conv1[(ft, f'feedback_to_item_{ft}', 'item')] = GATConv(emb_dim, hidden_dim,
                                                       heads=heads,
                                                       add_self_loops=False)
            conv2[(ft, f'feedback_to_item_{ft}', 'item')] = GATConv(hidden_dim*heads, emb_dim,
                                                       heads=1,
                                                       add_self_loops=False)

            # item -> ft
            conv1[('item', f'to_feedback_{ft}', ft)] = GATConv(emb_dim, hidden_dim,
                                                       heads=heads,
                                                       add_self_loops=False)
            conv2[('item', f'to_feedback_{ft}', ft)] = GATConv(hidden_dim*heads, emb_dim,
                                                       heads=1,
                                                       add_self_loops=False)

        self.conv1 = HeteroConv(conv1, aggr='mean')
        self.conv2 = HeteroConv(conv2, aggr='mean')

        # LayerNorm и Dropout
        types = ['user', 'item'] + feedback_types
        self.norm1 = nn.ModuleDict({t: nn.LayerNorm(hidden_dim*heads) for t in types})
        self.norm2 = nn.ModuleDict({t: nn.LayerNorm(emb_dim) for t in types})
        self.dropout = nn.Dropout(dropout)

    def forward(self, data):
        x = {
            'user': self.user_emb(data['user'].node_id),
            'item': self.item_emb(data['item'].node_id)
        }
        for ft in self.feedback_types:
            x[ft] = self.fb_emb[ft](data[ft].node_id)

        h1 = self.conv1(x, data.edge_index_dict)
        # print(h1.keys())
        # Собираем позитивный и негативный вклад для user
        pos_msgs = torch.zeros_like(h1['user'])
        neg_msgs = torch.zeros_like(h1['user'])
        for ft in self.pos_types:
            pos_msgs += h1[ft]  
        for ft in self.neg_types:
            neg_msgs += h1[ft]

        # signed-aggregation: attraction от pos, repulsion от neg
        user_h1 = pos_msgs - neg_msgs
        user_h1 = F.leaky_relu(self.norm1['user'](user_h1))
        user_h1 = self.dropout(user_h1)

        h1['user'] = user_h1

        h2 = self.conv2(h1, data.edge_index_dict)
        # print(h2.keys())
        pos_msgs2, neg_msgs2 = 0, 0
        for ft in self.pos_types:
            pos_msgs2 += h2[ft]
        for ft in self.neg_types:
            neg_msgs2 += h2[ft]

        user_h2 = pos_msgs2 - neg_msgs2
        user_h2 = F.leaky_relu(self.norm2['user'](user_h2))

        return user_h2

In [21]:
data = prepare_hetero_data(train)
data

HeteroData(
  user={ node_id=[18892] },
  item={ node_id=[25456] },
  implicit_negative={ node_id=[18892] },
  explicit_positive={ node_id=[18892] },
  implicit_positive={ node_id=[18892] },
  expliсit_negative={ node_id=[18892] },
  (item, to_feedback_implicit_negative, implicit_negative)={ edge_index=[2, 258651] },
  (implicit_negative, feedback_to_item_implicit_negative, item)={ edge_index=[2, 258651] },
  (implicit_negative, to_user_implicit_negative, user)={ edge_index=[2, 18892] },
  (item, to_feedback_explicit_positive, explicit_positive)={ edge_index=[2, 359463] },
  (explicit_positive, feedback_to_item_explicit_positive, item)={ edge_index=[2, 359463] },
  (explicit_positive, to_user_explicit_positive, user)={ edge_index=[2, 18892] },
  (item, to_feedback_implicit_positive, implicit_positive)={ edge_index=[2, 430342] },
  (implicit_positive, feedback_to_item_implicit_positive, item)={ edge_index=[2, 430342] },
  (implicit_positive, to_user_implicit_positive, user)={ edge_index

In [22]:
train.item_id.nunique(), train.item_id.min(), train.item_id.max()

(25456, 0, 25455)

In [23]:
num_users = len(train['user_id'].unique())
num_items = train['item_id'].max() + 1
feedback_types = train['target'].unique().tolist()
model = SignedHeteroGNN(num_users, num_items, feedback_types)

In [24]:
model

SignedHeteroGNN(
  (user_emb): Embedding(18892, 32)
  (item_emb): Embedding(25456, 32)
  (fb_emb): ModuleDict(
    (implicit_negative): Embedding(18892, 32)
    (explicit_positive): Embedding(18892, 32)
    (implicit_positive): Embedding(18892, 32)
    (expliсit_negative): Embedding(18892, 32)
  )
  (conv1): HeteroConv(num_relations=12)
  (conv2): HeteroConv(num_relations=12)
  (norm1): ModuleDict(
    (user): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (item): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (implicit_negative): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (explicit_positive): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (implicit_positive): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (expliсit_negative): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
  )
  (norm2): ModuleDict(
    (user): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (item): LayerNorm((32,), eps=1e-05, elementwise_affine=True)

In [25]:
model(data)

tensor([[ 1.5162e-01, -7.1993e-03, -1.1890e-02,  ..., -9.2425e-05,
         -4.1174e-03,  2.5627e-01],
        [ 3.5853e-01, -9.4277e-03, -2.6036e-03,  ..., -3.6192e-03,
         -1.2521e-02,  3.6283e-01],
        [-7.5855e-03, -7.2857e-03, -4.0534e-03,  ...,  1.1243e+00,
         -7.8982e-03,  6.9816e-01],
        ...,
        [ 4.1303e-01, -1.0085e-02, -1.1072e-02,  ..., -6.9892e-03,
          2.3222e-01, -1.1042e-03],
        [ 5.2519e-01, -1.3659e-02, -1.9419e-02,  ...,  8.0117e-01,
         -7.5687e-03,  1.7108e+00],
        [-2.6951e-03, -1.8021e-03, -6.7737e-03,  ..., -5.3802e-03,
          2.1237e-01, -5.6841e-04]], grad_fn=<LeakyReluBackward0>)

In [26]:
test_df = test[['user_id', 'item_id']]
interactions = test_df.rename(columns={
    'user_id': Columns.User,
    'item_id': Columns.Item,
})

viewed_items = train.groupby("user_id")["item_id"].agg(set).to_dict()

In [27]:
def evaluate(model, train_data,
             test_batch_size, top_k,
             viewed_items, interactions,
             device, test_step):
    """
    Оцениваем модель по всем пользователям:
    - строим топ-K рекомендации
    - фильтруем уже просмотренные
    - считаем recall@K, precision@K, map@K
    """
    model.eval()
    model.to(device)
    num_users = train_data['user'].node_id.shape[0]
    test_top_k = top_k * 150

    item_emb = model.item_emb.weight
    item_emb_t = item_emb.t().detach()
    del item_emb
    gc.collect()

    all_scores = []
    with torch.no_grad():
        for i in range(0, num_users, test_batch_size):
            end = min(i + test_batch_size, num_users)
            batch_users = torch.arange(i, end).to(device)
            user_e = model(
                data=train_data.to(device)
            )
            rating = torch.mm(user_e[batch_users].detach(), item_emb_t)
            _, topk = torch.topk(rating, k=test_top_k, dim=1)
            all_scores.append(topk)

            del user_e, rating
            gc.collect()
    all_scores = torch.cat(all_scores, dim=0).cpu().numpy()

    users_list, items, ranks = [], [], []
    for u in range(num_users):
        seen = viewed_items.get(u, set())
        recs = all_scores[u]
        mask = ~np.isin(recs, list(seen))
        filtered = recs[mask][:top_k]
        for rank, it in enumerate(filtered, 1):
            users_list.append(u)
            items.append(int(it))
            ranks.append(rank)
    reco_df = pd.DataFrame({
        'user_id': users_list,
        'item_id': items,
        'rank': ranks
    })

    metrics = {
        f'map@{top_k}': MAP(k=top_k),
        f'precision@{top_k}': Precision(k=top_k),
        f'recall@{top_k}': Recall(k=top_k),
        f'ndcg@{top_k}': NDCG(k=top_k)
    }
    results = calc_metrics(metrics=metrics,
                           reco=reco_df,
                           interactions=interactions)
    print(f"Step {test_step} — Test metrics:")
    for name, val in results.items():
        print(f"  {name}: {val:.9f}")
        experiment.log_metric(f"Test {name} vs step", val, step=test_step)
    del all_scores
    gc.collect()

    model.to(device)
    train_data.to(device)
    model.train()
    return results

In [36]:
def train_model(model,
                train_data: HeteroData,
                edge_type: tuple,
                num_epochs: int = 10,
                lr: float = 1e-3,
                batch_size: int = 1024,
                device: str = None,
                print_every: int = 100,
                test_every: int = 500,
                top_k: int = 10,
                test_batch_size=2048):
    device = device or ('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    train_data = train_data.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    src, dst = train_data[edge_type].edge_index
    num_train = src.size(0)
    test_top_k = top_k * 300
    total_steps = 5497
    print(f"Num of training examples: {num_train}")
    for epoch in range(501,1001):
        model.train()
        perm = torch.randperm(num_train, device=device)
        total_loss = 0.0
        step = 0
        for i in range(0, num_train, batch_size):
            idx = perm[i:i + batch_size]
            users = dst[idx]
            pos_items = src[idx]
            neg_items = torch.randint(0, model.item_emb.num_embeddings,
                                      size=pos_items.size(), device=device)
            optimizer.zero_grad()
            user_embs = model(train_data)[users]
            pos_emb = model.item_emb(pos_items)
            neg_emb = model.item_emb(neg_items)
            pos_score = (user_embs * pos_emb).sum(dim=1)
            neg_score = (user_embs * neg_emb).sum(dim=1)
            loss = -torch.log(torch.sigmoid(pos_score - neg_score) + 1e-15).mean()
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * users.size(0)
            step += 1

            experiment.log_metric('Train BPR Loss vs step', loss.item(), step=total_steps)
            if step % print_every == 0 or step == 1:
                avg_loss = total_loss /  (step * batch_size)
                current_lr = optimizer.param_groups[0]['lr']
                d = (pos_score - neg_score).detach().cpu()
                print(f"Epoch {epoch}, Step {step}, LR: {current_lr:.6f}, Current Loss: {loss.item():.4f}, Avg Loss: {avg_loss:.4f}")
                print(f"Diff stats — min: {d.min():.4f}, max: {d.max():.4f}, mean: {d.mean():.4f}, std: {d.std():.4f}")
                print()

                experiment.log_metric('Diff stats (mean) vs step', d.mean(), step=total_steps)
                experiment.log_metric('Diff stats (std) vs step', d.std(), step=total_steps)

            del user_embs, pos_emb, neg_emb, pos_score, neg_score
            gc.collect()
            torch.cuda.empty_cache()
            
            if step % test_every == 0 or step == 1:
                evaluate(model, train_data,
                         test_batch_size, top_k,
                         viewed_items, interactions,
                         device, test_step=total_steps)
            total_steps += 1
        epoch_loss = total_loss / num_train
        experiment.log_metric(f'Train Loss vs epoch', epoch_loss, epoch=epoch - 1)
        print(f"Epoch {epoch} completed, Train Loss: {epoch_loss:.4f}")
    return model

In [29]:
experiment.log_parameters(hyperparameters)

In [30]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [31]:
edge_type = hyperparameters['train_edge_type']
num_epochs = hyperparameters['train_num_epochs']
lr = hyperparameters['train_lr']
batch_size = hyperparameters['train_batch_size']
print_every = hyperparameters['train_print_every']
test_every = hyperparameters['train_test_every']
top_k = hyperparameters['test_topk']
test_batch_size = hyperparameters['test_batch_size']
model = train_model(model,
                    data,
                    edge_type=edge_type,
                    num_epochs=num_epochs,
                    lr=lr,
                    batch_size=batch_size,
                    print_every=print_every,
                    test_every=test_every,
                    top_k=top_k,
                    test_batch_size=test_batch_size)

Num of training examples: 359463
Epoch 1, Step 1, LR: 0.000080, Current Loss: 2.3516, Avg Loss: 2.3516
Diff stats — min: -23.6654, max: 24.3270, mean: 0.0091, std: 5.6245

Step 0 — Test metrics:
  precision@10: 0.000396406
  recall@10: 0.000397874
  ndcg@10: 0.000407474
  map@10: 0.000124084
Epoch 1, Step 10, LR: 0.000080, Current Loss: 2.2534, Avg Loss: 2.3044
Diff stats — min: -20.5828, max: 21.5892, mean: 0.0318, std: 5.3981

Step 9 — Test metrics:
  precision@10: 0.000383192
  recall@10: 0.000384661
  ndcg@10: 0.000375116
  map@10: 0.000108172
Epoch 1 completed, Train Loss: 2.2995
Epoch 2, Step 1, LR: 0.000080, Current Loss: 2.2087, Avg Loss: 2.2087
Diff stats — min: -26.2154, max: 21.1165, mean: 0.0978, std: 5.3705

Step 11 — Test metrics:
  precision@10: 0.000383192
  recall@10: 0.000384661
  ndcg@10: 0.000375140
  map@10: 0.000108199
Epoch 2, Step 10, LR: 0.000080, Current Loss: 2.1678, Avg Loss: 2.2084
Diff stats — min: -22.4431, max: 22.2424, mean: 0.0200, std: 5.1674

Step 20

In [33]:
model = train_model(model,
                    data,
                    edge_type=edge_type,
                    num_epochs=num_epochs,
                    lr=lr,
                    batch_size=batch_size,
                    print_every=print_every,
                    test_every=test_every,
                    top_k=top_k,
                    test_batch_size=test_batch_size)

Num of training examples: 359463
Epoch 101, Step 1, LR: 0.000080, Current Loss: 0.8119, Avg Loss: 0.8119
Diff stats — min: -11.2890, max: 12.3440, mean: 0.2172, std: 1.4980

Step 1099 — Test metrics:
  precision@10: 0.000601216
  recall@10: 0.000601950
  ndcg@10: 0.000571621
  map@10: 0.000158384
Epoch 101, Step 10, LR: 0.000080, Current Loss: 0.8067, Avg Loss: 0.8087
Diff stats — min: -9.3998, max: 12.5838, mean: 0.2213, std: 1.4834

Step 1108 — Test metrics:
  precision@10: 0.000588002
  recall@10: 0.000588736
  ndcg@10: 0.000563141
  map@10: 0.000156840
Epoch 101 completed, Train Loss: 0.8085
Epoch 102, Step 1, LR: 0.000080, Current Loss: 0.8116, Avg Loss: 0.8116
Diff stats — min: -11.5500, max: 12.2888, mean: 0.2156, std: 1.4992

Step 1110 — Test metrics:
  precision@10: 0.000601216
  recall@10: 0.000601950
  ndcg@10: 0.000572786
  map@10: 0.000158704
Epoch 102, Step 10, LR: 0.000080, Current Loss: 0.8044, Avg Loss: 0.8047
Diff stats — min: -7.9543, max: 11.3144, mean: 0.2130, std:

In [35]:
model = train_model(model,
                    data,
                    edge_type=edge_type,
                    num_epochs=num_epochs,
                    lr=lr,
                    batch_size=batch_size,
                    print_every=print_every,
                    test_every=test_every,
                    top_k=top_k,
                    test_batch_size=test_batch_size)

Num of training examples: 359463
Epoch 201, Step 1, LR: 0.000080, Current Loss: 0.6295, Avg Loss: 0.6295
Diff stats — min: -5.0975, max: 6.5454, mean: 0.2994, std: 0.8144

Step 2198 — Test metrics:
  precision@10: 0.001427061
  recall@10: 0.001428529
  ndcg@10: 0.001521570
  map@10: 0.000472687
Epoch 201, Step 10, LR: 0.000080, Current Loss: 0.6268, Avg Loss: 0.6289
Diff stats — min: -7.8867, max: 7.3657, mean: 0.3024, std: 0.8066

Step 2207 — Test metrics:
  precision@10: 0.001479915
  recall@10: 0.001481384
  ndcg@10: 0.001591277
  map@10: 0.000497965
Epoch 201 completed, Train Loss: 0.6288
Epoch 202, Step 1, LR: 0.000080, Current Loss: 0.6287, Avg Loss: 0.6287
Diff stats — min: -4.2175, max: 7.3493, mean: 0.2994, std: 0.8119

Step 2209 — Test metrics:
  precision@10: 0.001460095
  recall@10: 0.001461563
  ndcg@10: 0.001577656
  map@10: 0.000495773
Epoch 202, Step 10, LR: 0.000080, Current Loss: 0.6285, Avg Loss: 0.6274
Diff stats — min: -4.5263, max: 14.7675, mean: 0.2990, std: 0.81

In [37]:
model = train_model(model,
                    data,
                    edge_type=edge_type,
                    num_epochs=num_epochs,
                    lr=lr,
                    batch_size=batch_size,
                    print_every=print_every,
                    test_every=test_every,
                    top_k=top_k,
                    test_batch_size=test_batch_size)

Num of training examples: 359463
Epoch 501, Step 1, LR: 0.000080, Current Loss: 0.3993, Avg Loss: 0.3993
Diff stats — min: -5.6972, max: 11.0533, mean: 1.5775, std: 1.7362

Step 5497 — Test metrics:
  precision@10: 0.006031977
  recall@10: 0.006034179
  ndcg@10: 0.006330427
  map@10: 0.001993567
Epoch 501, Step 10, LR: 0.000080, Current Loss: 0.4030, Avg Loss: 0.3996
Diff stats — min: -6.4241, max: 10.5434, mean: 1.5768, std: 1.7500

Step 5506 — Test metrics:
  precision@10: 0.005992336
  recall@10: 0.005996007
  ndcg@10: 0.006302921
  map@10: 0.001986033
Epoch 501 completed, Train Loss: 0.3993
Epoch 502, Step 1, LR: 0.000080, Current Loss: 0.4035, Avg Loss: 0.4035
Diff stats — min: -8.8749, max: 8.3995, mean: 1.5696, std: 1.7410

Step 5508 — Test metrics:
  precision@10: 0.005979123
  recall@10: 0.005982793
  ndcg@10: 0.006259778
  map@10: 0.001963271
Epoch 502, Step 10, LR: 0.000080, Current Loss: 0.3991, Avg Loss: 0.3988
Diff stats — min: -6.0197, max: 9.0108, mean: 1.5832, std: 1.7

In [33]:
torch.save(model, "gnn_model_mvl.model")
from IPython.display import FileLink

FileLink('gnn_model_mvl.model')

/kaggle/working/gnn_model_mvl.model

In [34]:
# del model
gc.collect()
torch.cuda.empty_cache()

In [35]:
log_model(
    experiment=experiment,
    model=model,
    model_name="GNN",
)

In [36]:
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : diffGATGraphModel-movielens
COMET INFO:     url                   : https://www.comet.com/annanet/gnn-recommender/527798220a1140c5b8dfc2d8f2ce1335
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     Diff stats (mean) vs step [1166] : (-0.2983660101890564, 2.9147939682006836)
COMET INFO:     Diff stats (std) vs step [1166]  : (0.8144240379333496, 5.913432598114014)
COMET INFO:     Test map@10 vs step [1166]       : (0.0017786384421318198, 0.013094419040611092)
COMET INFO:     Test ndcg@10 vs step [1166]      : (0.005354564317200823, 0.03636796341204262)
COMET INFO:     Test precision@10 vs step [1166] : (0.004668874172185431, 0.03586092715231788